In [ ]:
import numpy as np, zarr, nrrd
from CircuitSeeker.function import distributed_deltafoverf

In [ ]:

# compute dff
dff = distributed_deltafoverf(
    './motion_corrected.zarr', 32, 100,
    './deltafoverf.zarr',
    cluster_kwargs={
        'ncpus':8, 'threads':1,
        'min_workers':50, 'max_workers':50,
        'config':{
            'distributed.worker.memory.target':0.7,
            'distributed.worker.memory.spill':False,
            'distributed.worker.memory.pause':0.9,
            'distributed.worker.memory.terminate':0.95,
        },
    },
)

In [ ]:
# how to slice in time/space
plane = 27
stride = 1

# initialize an array to hold slices
slice_over_time = np.empty((3861, 1224, 2048), dtype=np.uint8)

# read all data
for i in range(3861):
    if i % 100 == 0: print(i)
    frame = dff[i*stride, plane, ...].astype(np.float32)
    mn, mx = frame.min(), frame.max()
    remapped = 255 * (frame - mn) / (mx - mn)
    slice_over_time[i] = np.round(remapped).astype(np.uint8)

# write out in a format you can read with Fiji/Icy etc.
nrrd.write('./slice27_dff_uint8.nrrd', slice_over_time.transpose(2,1,0), compression_level=2)